In [ ]:
# upgrade version of altair from 4.2.0 to 5.0.0
%pip install altair==5.0.0rc1

In [ ]:
import altair as alt
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv')
print(alt.__version__)
myJekyllDir = '../assets/json/'
df.columns

In [ ]:
brush = alt.selection_interval()

heatmap = alt.Chart('https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv').mark_rect().encode(
    x=alt.X('wind_speed:Q', bin=True),
    y=alt.Y('state:N'),
    color=alt.Color('count(state):Q', scale=alt.Scale(scheme='Blues'))
).properties(
    height=500
).add_params(
    brush
)

bar_chart = alt.Chart.from_dict({
  "description": "Mean number of ufos per state",
  "data": {"url": "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
  },
  "height":400,
  "width":500,
  "mark": "bar",
  "encoding": {
    "x": {"field": "state", "type": "nominal"},
    "y": {"aggregate":"mean", "field":"number", "type":"quantitative",
      "axis": {
        "title": "mean number of ufos"
      }
    }
  }
}).transform_filter(
    brush
)

In [ ]:
dashbord = heatmap.properties(width=150) | bar_chart.properties(width=500)
dashbord

In [ ]:
dashbord.save(myJekyllDir + 'IS445_assignment10_ufo_dashboard.json')

In [ ]:
input_dropdown = alt.binding_select(options=df['state'].unique(), name='Region')
selection = alt.selection_point(fields=['state'], bind=input_dropdown)
color = alt.condition(
    selection,
    alt.Color('state:N').legend(None),
    alt.value('lightgray')
)

us_map = alt.Chart.from_dict({
    "width": 800,
    "height": 500,
    "projection": {
        "type": "albersUsa"
    },
    "data": {
        "url": "https://raw.githubusercontent.com/vega/vega-datasets/master/data/us-10m.json",
        "format": {
            "type": "topojson",
            "feature": "states"
        }
    },
    "mark": {
        "type": "geoshape",
        "fill": "lightgray",
        "stroke": "white"
    }
})

sightings_map = alt.Chart.from_dict({
    "data": {
        "url": "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
    },
    "encoding": {
        "longitude": {
            "field": "longitude",
            "type": "quantitative"
        },
        "latitude": {
            "field": "latitude",
            "type": "quantitative"
        }
    },
    "mark": {
        "type": "circle",
    }
}).mark_point().encode(
    color=color,
).add_params(
    selection
)

map_chart = us_map + sightings_map
map_chart

In [ ]:
map_chart.properties(width='container').save(myJekyllDir + 'IS445_assignment10_ufo_sightings_map.json')